## Inventory Optimization with Data Science: Hands-On Tutorial with Python

### Part 2: A Gentle Introduction to Implementing the Markov Process for Inventory Optimization.

In [31]:
from typing import Dict

from rich import pretty
pretty.install()

# need numpy to do some numeric calculation
import numpy as np

# poisson is used to find pdf of Poisson distribution 
from scipy.stats import poisson

import pandas as pd
import numpy as np

MarkovRewProcessDict = {"Current State A":{"NextS1fromA": ("PNextS1fromA","Reward1")
                                           ,"NextS2fromA, from A": ("PNextS2fromA","Reward2")},
                    
                     "Current State B":{"NextS1fromB": ("PNextS1fromB","Reward3"),
                                        "NextS2fromB": ("PNextS2fromB","Reward4")}}

MarkovRewProcessDict

{
    'Current State A': {
        'NextS1fromA': ('PNextS1fromA', 'Reward1'),
        'NextS2fromA, from A': ('PNextS2fromA', 'Reward2')
    },
    'Current State B': {
        'NextS1fromB': ('PNextS1fromB', 'Reward3'),
        'NextS2fromB': ('PNextS2fromB', 'Reward4')
    }
}

In [37]:
for (state, value) in MarkovRewProcessDict.items():
    
    print("The Current state is: {}".format(state))
    
    for (next_state, (trans_prob,reward)) in value.items():
        
        print("The Next State is {} \nwith Probability of: {} \
               \nand Reward: {}".format(next_state, trans_prob, reward))
    

The Current state is: (0, 0)
The Next State is (0, 2) 
with Probability of: 1.0                
and Reward: -10.0
The Current state is: (0, 1)
The Next State is (1, 1) 
with Probability of: 0.3678794411714424                
and Reward: 0
The Next State is (0, 1) 
with Probability of: 0.6321205588285576                
and Reward: -3.6787944117144225
The Current state is: (0, 2)
The Next State is (2, 0) 
with Probability of: 0.3678794411714424                
and Reward: 0
The Next State is (1, 0) 
with Probability of: 0.3678794411714424                
and Reward: 0
The Next State is (0, 0) 
with Probability of: 0.26424111765711533                
and Reward: -1.03638323514327
The Current state is: (1, 0)
The Next State is (1, 1) 
with Probability of: 0.3678794411714424                
and Reward: -1
The Next State is (0, 1) 
with Probability of: 0.6321205588285576                
and Reward: -4.6787944117144225
The Current state is: (1, 1)
The Next State is (2, 0) 
with Probability o

## Reward Modeling (two cases)

- Case for 
- $0 \leq i \leq \alpha + \beta -1$ 

$$R((\alpha , \beta) \to ((\alpha + \beta) -i, C-(\alpha + \beta) ) = -h\alpha$$

- For the case demand excedding $(\alpha + \beta)$

- $$R((\alpha , \beta) \to (0, C-(\alpha + \beta) ) = -h\alpha -p(\sum_{j=\alpha + \beta +1}^{\infin}f(j)(j-(\alpha +\beta)))$$
- $$=h\alpha-p(\lambda(1-F(\alpha+\beta-1)) - (\alpha+\beta)(1-F(\alpha+\beta)))$$

In [65]:

MarkovRewProcessDict: Dict[tuple, Dict[tuple, tuple]] = {}

user_capacity = 2
user_poisson_lambda = 1.0

holding_cost = 1
stockout_cost = 10

gamma = 0.9 


In [39]:
# We are condiering all possible states
# That we can face in running this bike shop
for alpha in range(user_capacity+1):                            
                                                               
    for beta in range(user_capacity + 1 - alpha):
        
        # This is St, the current state
        state = (alpha, beta)                                   

        # This is initial inventory, total bike you have at 8AM 
        initial_inventory = alpha + beta                         
        
        # The beta1 is the beta in next state, irrespctive of current state (as the decsion policy is constant)
        beta1 = user_capacity - initial_inventory
        
        base_reward = -alpha* holding_cost
        # List of all possible demand you can get
        for i in range(initial_inventory +1):

            # if initial demand can meet the demand
            if i <= (initial_inventory-1):
                
                # probality of specifc demand can happen
                transition_prob = poisson.pmf(i,user_poisson_lambda)
                
                # If we already defined teh state in our data (MarkovProcessDict)
                if state in MarkovRewProcessDict:
                    
                    MarkovRewProcessDict[state][(initial_inventory - i, beta1)]= (transition_prob, base_reward)
                
                else:
                    
                    MarkovRewProcessDict[state] = {(initial_inventory - i, beta1):(transition_prob, base_reward)}
                         
            # if initial demand can not meet the demand
            else:
                
                transition_prob = 1- poisson.cdf(initial_inventory -1, user_poisson_lambda)
                # probability of not meeting the demands
                #transition_prob1 = 1- poisson.cdf(initial_inventory -1, user_poisson_lambda)
                
                # probability of not meeting the demands
                transition_prob2 = 1- poisson.cdf(initial_inventory, user_poisson_lambda)
                
                # total reward
                
                reward = base_reward - stockout_cost*((user_poisson_lambda*transition_prob) - \
                        initial_inventory*transition_prob2)
                #print(rewardf)
                

                #reward = base_reward - stockout_cost *(user_poisson_lambda - initial_inventory *(
                #    1 - poisson.pmf(initial_inventory, user_poisson_lambda)/transition_prob1))
                #print(reward)
                if state in MarkovRewProcessDict:
                    
                    MarkovRewProcessDict[state][(0, beta1)]= (transition_prob,reward)
                    
                else:

                    MarkovRewProcessDict[state] = {(0, beta1):(transition_prob, reward)}

In [35]:
MarkovRewProcessDict

{
    (0, 0): {(0, 2): (1.0, -10.0)},
    (0, 1): {
        (1, 1): (0.3678794411714424, 0),
        (0, 1): (0.6321205588285576, -3.6787944117144225)
    },
    (0, 2): {
        (2, 0): (0.3678794411714424, 0),
        (1, 0): (0.3678794411714424, 0),
        (0, 0): (0.26424111765711533, -1.03638323514327)
    },
    (1, 0): {
        (1, 1): (0.3678794411714424, -1),
        (0, 1): (0.6321205588285576, -4.6787944117144225)
    },
    (1, 1): {
        (2, 0): (0.3678794411714424, -1),
        (1, 0): (0.3678794411714424, -1),
        (0, 0): (0.26424111765711533, -2.03638323514327)
    },
    (2, 0): {
        (2, 0): (0.3678794411714424, -2),
        (1, 0): (0.3678794411714424, -2),
        (0, 0): (0.26424111765711533, -3.03638323514327)
    }
}

In [36]:
for (state, value) in MarkovRewProcessDict.items():
    
    print("The Current state is: {}".format(state))
    
    for (next_state, (trans_prob,reward)) in value.items():
        
        print("The Next State is {} \nwith Probability of: {} \
               \nand Reward: {}".format(next_state, trans_prob, reward))

The Current state is: (0, 0)
The Next State is (0, 2) 
with Probability of: 1.0                
and Reward: -10.0
The Current state is: (0, 1)
The Next State is (1, 1) 
with Probability of: 0.3678794411714424                
and Reward: 0
The Next State is (0, 1) 
with Probability of: 0.6321205588285576                
and Reward: -3.6787944117144225
The Current state is: (0, 2)
The Next State is (2, 0) 
with Probability of: 0.3678794411714424                
and Reward: 0
The Next State is (1, 0) 
with Probability of: 0.3678794411714424                
and Reward: 0
The Next State is (0, 0) 
with Probability of: 0.26424111765711533                
and Reward: -1.03638323514327
The Current state is: (1, 0)
The Next State is (1, 1) 
with Probability of: 0.3678794411714424                
and Reward: -1
The Next State is (0, 1) 
with Probability of: 0.6321205588285576                
and Reward: -4.6787944117144225
The Current state is: (1, 1)
The Next State is (2, 0) 
with Probability o

In [55]:
# Extract unique state keys
states = list(MarkovRewProcessDict.keys())

# Initialize an empty matrix
trans_prob = np.zeros((len(states), len(states)))

# Populate the matrix with probabilities
for i, from_state in enumerate(states):
    for j, to_state in enumerate(states):
        if to_state in MarkovRewProcessDict[from_state]:
            probability, _ = MarkovRewProcessDict[from_state][to_state]
            trans_prob[i, j] = probability

# Create a DataFrame
df_trans_prob = pd.DataFrame(trans_prob, columns=states, index=states)

# Print the DataFrame
df_trans_prob

,"(0, 0)","(0, 1)","(0, 2)","(1, 0)","(1, 1)","(2, 0)"
"(0, 0)",0.000000,0.000000,1.0,0.000000,0.000000,0.000000
"(0, 1)",0.000000,0.632121,0.0,0.000000,0.367879,0.000000
"(0, 2)",0.264241,0.000000,0.0,0.367879,0.000000,0.367879
"(1, 0)",0.000000,0.632121,0.0,0.000000,0.367879,0.000000
"(1, 1)",0.264241,0.000000,0.0,0.367879,0.000000,0.367879
"(2, 0)",0.264241,0.000000,0.0,0.367879,0.000000,0.367879


In [42]:
MarkovRewProcessDict

{
    (0, 0): {(0, 2): (1.0, -10.0)},
    (0, 1): {
        (1, 1): (0.3678794411714424, 0),
        (0, 1): (0.6321205588285576, -3.6787944117144225)
    },
    (0, 2): {
        (2, 0): (0.3678794411714424, 0),
        (1, 0): (0.3678794411714424, 0),
        (0, 0): (0.26424111765711533, -1.03638323514327)
    },
    (1, 0): {
        (1, 1): (0.3678794411714424, -1),
        (0, 1): (0.6321205588285576, -4.6787944117144225)
    },
    (1, 1): {
        (2, 0): (0.3678794411714424, -1),
        (1, 0): (0.3678794411714424, -1),
        (0, 0): (0.26424111765711533, -2.03638323514327)
    },
    (2, 0): {
        (2, 0): (0.3678794411714424, -2),
        (1, 0): (0.3678794411714424, -2),
        (0, 0): (0.26424111765711533, -3.03638323514327)
    }
}

In [46]:
def expected_immediateR(data):
    
    expected_value = 0
    
    for key, value in data.items():
            weight, val = value
            expected_value += weight * val
    
    return expected_value


E_immediate_R = {}

for key, value in MarkovRewProcessDict.items():
    E_immediate_R[key] = expected_immediateR(value)

E_immediate_R

{
    (0, 0): -10.0,
    (0, 1): -2.325441579348295,
    (0, 2): -0.2738550643753546,
    (1, 0): -3.325441579348295,
    (1, 1): -1.2738550643753548,
    (2, 0): -2.273855064375355
}

$$f(j)=\frac{e^{-\lambda}\lambda^{j}}{j!}$$

$$F(x)=\sum_{j=0}^{x}\frac{e^{-\lambda}\lambda^{j}}{j!}$$

$$\mathbb{E} [j \leq \alpha +\beta ] = \sum_{j=0}^{\alpha + \beta}\frac{e^{-\lambda}\lambda^{j}}{j!}j$$

$$\mathbb{E} [j \leq \alpha +\beta ] = \sum_{j=1}^{\alpha + \beta}\frac{e^{-\lambda}\lambda^{j}}{j!}j= \sum_{j=1}^{\alpha + \beta}\frac{e^{-\lambda}\lambda^{j}}{j(j-1)!}j=\sum_{j=1}^{\alpha + \beta}\frac{e^{-\lambda}\lambda^{j}}{(j-1)!} = \lambda \sum_{j=1}^{\alpha + \beta}\frac{e^{-\lambda}\lambda^{j-1}}{(j-1)!}$$
$$\mathbb{E} [j \leq \alpha +\beta ] = \lambda \sum_{k=0}^{\alpha + \beta -1}\frac{e^{-\lambda}\lambda^{k}}{k!} =\lambda F(\alpha + \beta -1)$$

In [54]:
R_exp=np.array(list(E_immediate_R.values()))
R_exp

array([-10.        ,  -2.32544158,  -0.27385506,  -3.32544158,
        -1.27385506,  -2.27385506])

In [57]:
trans_prob

array([[0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.63212056, 0.        , 0.        , 0.36787944,
        0.        ],
       [0.26424112, 0.        , 0.        , 0.36787944, 0.        ,
        0.36787944],
       [0.        , 0.63212056, 0.        , 0.        , 0.36787944,
        0.        ],
       [0.26424112, 0.        , 0.        , 0.36787944, 0.        ,
        0.36787944],
       [0.26424112, 0.        , 0.        , 0.36787944, 0.        ,
        0.36787944]])

In [71]:
val_func_vec = np.linalg.solve(np.eye(len(states)) - gamma*trans_prob, R_exp)
val_func_vec

array([-35.51060433, -27.93226038, -28.34511593, -28.93226038,
       -29.34511593, -30.34511593])

In [70]:
R_exp

array([-10.        ,  -2.32544158,  -0.27385506,  -3.32544158,
        -1.27385506,  -2.27385506])

In [77]:
MarkRevData = pd.DataFrame({'Expected Immediate Reward': R_exp, 'Value Function': val_func_vec}, index=states)
MarkRevData

,Expected Immediate Reward,Value Function
"(0, 0)",-10.000000,-35.510604
"(0, 1)",-2.325442,-27.932260
"(0, 2)",-0.273855,-28.345116
"(1, 0)",-3.325442,-28.932260
"(1, 1)",-1.273855,-29.345116
"(2, 0)",-2.273855,-30.345116


- This computation for the Value Function works if the state space is not too large.
- Whe the state space is large, this direct method of solving a linear system of equations will not scale.
- We need to resort to numerical methods (DP, RL)